# Decision trees and NN tryouts on SPR data

Load training and validation data as 
    month : [ Features | Targets| Difference | Last Choice Targets  ]
    

In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd

import logging
logging.getLogger().handlers = []
logging.basicConfig(level=logging.INFO)

import matplotlib.pylab as plt
%matplotlib inline

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [24]:
import sys
sys.path.append("../common")

from dataset2 import load_trainval, FEATURES_NAMES, TARGET_GROUP_DEC_LABELS, TARGET_LABELS_FRQ
from utils import to_yearmonth, TARGET_LABELS, TARGET_LABELS2
from utils import target_str_to_labels, decimal_to_dummies, targets_str_to_indices, targets_dec_to_indices

In [66]:
df = load_trainval(201605, 1500)

INFO:root:- Load file : ../data/train_ver2.csv, yearmonth=201605, n_clients=1500
INFO:root:-- Select 1500 clients
INFO:root:- Number of lines with unknown data : 0
INFO:root:- Number of columns with nan : 9
INFO:root:-- Process targets for one yearmonth : 201605
INFO:root:-- Process features for one yearmonth : 201605
INFO:root:-- Process targets for one yearmonth : 201604
INFO:root:-- Process features for one yearmonth : 201604


['ind_valo_fin_ult1_frq', 'targets_dec_g3_1', 'targets_str_1', 'ind_ctpp_fin_ult1_frq_1', 'ind_ahor_fin_ult1_frq', 'ind_plan_fin_ult1_frq_1', 'ind_nom_pens_ult1_frq', 'ind_pres_fin_ult1_frq_1', 'ind_deme_fin_ult1_frq_1', 'ind_viv_fin_ult1_frq_1', 'ind_aval_fin_ult1_frq_1', 'ind_recibo_ult1_frq_1', 'ind_ctop_fin_ult1_frq', 'ind_ctma_fin_ult1_frq', 'ind_ctju_fin_ult1_frq_1', 'targets_str', 'ind_ctop_fin_ult1_frq_1', 'targets_dec_g0_1', 'ind_cder_fin_ult1_frq', 'ind_deme_fin_ult1_frq', 'targets_dec_g1_1', 'ind_ctma_fin_ult1_frq_1', 'ind_pres_fin_ult1_frq', 'targets_dec_g2_1', 'ind_cder_fin_ult1_frq_1', 'targets_logdec_1', 'ind_ctpp_fin_ult1_frq', 'ind_viv_fin_ult1_frq', 'ind_dela_fin_ult1_frq_1', 'ind_cco_fin_ult1_frq_1', 'ind_recibo_ult1_frq', 'ind_valo_fin_ult1_frq_1', 'ind_cno_fin_ult1_frq_1', 'ind_reca_fin_ult1_frq_1', 'ind_deco_fin_ult1_frq', 'targets_dec_g1', 'targets_dec_g0', 'targets_dec_g3', 'targets_dec_g2', 'ind_hip_fin_ult1_frq', 'ind_fond_fin_ult1_frq_1', 'ind_fond_fin_ult1_f

In [67]:
df.shape

(1500, 106)

In [68]:
df[['ncodpers', 'fecha_dato', 'targets_str', 'targets_str_1', 'ind_cco_fin_ult1_frq', 'ind_cder_fin_ult1_frq', 'ind_cco_fin_ult1_frq_1', 'ind_cder_fin_ult1_frq_1']].head(10)

,ncodpers,fecha_dato,targets_str,targets_str_1,ind_cco_fin_ult1_frq,ind_cder_fin_ult1_frq,ind_cco_fin_ult1_frq_1,ind_cder_fin_ult1_frq_1
310589,16056,2016-05-28,NaN,001010001000000000000001,NaN,NaN,0.618667,0.998667
310974,16926,2016-05-28,NaN,000000000000000000000000,NaN,NaN,0.381333,0.998667
310714,17147,2016-05-28,NaN,000000000000000000000000,NaN,NaN,0.381333,0.998667
312384,19775,2016-05-28,NaN,001000000000010000000000,NaN,NaN,0.618667,0.998667
312864,20565,2016-05-28,NaN,001000000000000000110000,NaN,NaN,0.618667,0.998667
311971,22398,2016-05-28,NaN,001000000000000001000000,NaN,NaN,0.618667,0.998667
311705,22955,2016-05-28,NaN,000000000000000000000000,NaN,NaN,0.381333,0.998667
306078,25232,2016-05-28,NaN,001000000000000000000000,NaN,NaN,0.618667,0.998667
305666,25688,2016-05-28,NaN,001000010000000001100000,NaN,NaN,0.618667,0.998667
306262,26781,2016-05-28,NaN,000000000000000000000000,NaN,NaN,0.381333,0.998667


In [20]:
print len(df.columns), df.columns.values

111 ['fecha_dato' 'ncodpers' 'ind_empleado' 'pais_residencia' 'sexo' 'age'
 'fecha_alta' 'ind_nuevo' 'antiguedad' 'indrel' 'ult_fec_cli_1t'
 'indrel_1mes' 'tiprel_1mes' 'indresi' 'indext' 'conyuemp' 'canal_entrada'
 'indfall' 'nomprov' 'ind_actividad_cliente' 'renta' 'segmento'
 'ind_ahor_fin_ult1' 'ind_aval_fin_ult1' 'ind_cco_fin_ult1'
 'ind_cder_fin_ult1' 'ind_cno_fin_ult1' 'ind_ctju_fin_ult1'
 'ind_ctma_fin_ult1' 'ind_ctop_fin_ult1' 'ind_ctpp_fin_ult1'
 'ind_deco_fin_ult1' 'ind_deme_fin_ult1' 'ind_dela_fin_ult1'
 'ind_ecue_fin_ult1' 'ind_fond_fin_ult1' 'ind_hip_fin_ult1'
 'ind_plan_fin_ult1' 'ind_pres_fin_ult1' 'ind_reca_fin_ult1'
 'ind_tjcr_fin_ult1' 'ind_valo_fin_ult1' 'ind_viv_fin_ult1'
 'ind_nomina_ult1' 'ind_nom_pens_ult1' 'ind_recibo_ult1' 'targets_str'
 'targets_logdec' 'targets_dec_g0' 'targets_dec_g1' 'targets_dec_g2'
 'targets_dec_g3' 'ind_ahor_fin_ult1_frq' 'ind_aval_fin_ult1_frq'
 'ind_cco_fin_ult1_frq' 'ind_cder_fin_ult1_frq' 'ind_cno_fin_ult1_frq'
 'ind_ctju_fin_ult1_f

In [71]:
X_train = load_X(201605, [201505, 201604], 150000)
# X, X_ym = load_X(201605, [201505, 201604], 1500)

INFO:root:- Load file : ../data/train_ver2.csv, yearmonth=201605, n_clients=5500
INFO:root:-- Select 5500 clients
INFO:root:- Number of lines with unknown data : 0
INFO:root:- Number of columns with nan : 9
INFO:root:- Process targets for one yearmonth : 201605
INFO:root:- Add a supplementary data : 201505
INFO:root:- Number of lines with unknown data : 3597
INFO:root:- Number of columns with nan : 10
INFO:root:- Compute missing clients : 1802/5500
INFO:root:- Match and add
INFO:root:-- Match and add elapsed time: 13.2841827869
INFO:root:- Compute missing clients : 1564/5500
INFO:root:- Match and add
INFO:root:-- Match and add elapsed time: 30.4563500881
INFO:root:- Compute missing clients : 945/5500
INFO:root:- Match and add
INFO:root:-- Match and add elapsed time: 38.723692894
INFO:root:- Compute missing clients : 155/5500
INFO:root:- Match and add
INFO:root:-- Match and add elapsed time: 4.82230901718
INFO:root:- Compute missing clients : 63/5500
INFO:root:- Match and add
INFO:root:

In [73]:
X_train.columns.values

array(['fecha_dato', 'ncodpers', 'ind_empleado', 'pais_residencia', 'sexo',
       'age', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel',
       'ult_fec_cli_1t', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext',
       'conyuemp', 'canal_entrada', 'indfall', 'nomprov',
       'ind_actividad_cliente', 'renta', 'segmento', 'ind_ahor_fin_ult1',
       'ind_aval_fin_ult1', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1',
       'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1',
       'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1',
       'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 'ind_ecue_fin_ult1',
       'ind_fond_fin_ult1', 'ind_hip_fin_ult1', 'ind_plan_fin_ult1',
       'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1',
       'ind_valo_fin_ult1', 'ind_viv_fin_ult1', 'ind_nomina_ult1',
       'ind_nom_pens_ult1', 'ind_recibo_ult1', 'targets_str',
       'targets_logdec', 'targets_dec_g0', 'targets_dec_g1',
       'targets_dec_g2', 'targets_dec_g3', '

In [75]:
X_train[['targets_str', 'targets_str_1', 'targets_str_2']].head()

,targets_str,targets_str_1,targets_str_2
310903,001000000000000000100000,001000000000000000100000,001000000000000000100000
310729,001000000000000001000001,001000000000000001000001,001000000000000001000001
310670,000000000000000000000000,001100011000000000000000,000000000000000000000000
309359,000010000000100001100111,000010000000100001000111,000010000000100001100111
309440,001000000000000100010000,001000000000000100010000,001000000000000100010000


In [18]:
X[['fecha_dato', 'ncodpers']].sort_values(['fecha_dato', 'ncodpers']).head()

,fecha_dato,ncodpers
310503,2016-05-28,15936
309780,2016-05-28,18987
309739,2016-05-28,19021
311087,2016-05-28,19299
312469,2016-05-28,20842


In [19]:
X_ym[['fecha_dato', 'ncodpers']].sort_values(['fecha_dato', 'ncodpers']).head()

,fecha_dato,ncodpers
628533,2015-05-28,15936
418150,2015-05-28,18987
418144,2015-05-28,19021
418393,2015-05-28,19299
418996,2015-05-28,20842


In [20]:
(X['ncodpers'] == X_ym['ncodpers']).all()

True

In [22]:
X.columns

Index([u'fecha_dato', u'ncodpers', u'ind_empleado', u'pais_residencia',
       u'sexo', u'age', u'fecha_alta', u'ind_nuevo', u'antiguedad', u'indrel',
       u'ult_fec_cli_1t', u'indrel_1mes', u'tiprel_1mes', u'indresi',
       u'indext', u'conyuemp', u'canal_entrada', u'indfall', u'nomprov',
       u'ind_actividad_cliente', u'renta', u'segmento', u'ind_ahor_fin_ult1',
       u'ind_aval_fin_ult1', u'ind_cco_fin_ult1', u'ind_cder_fin_ult1',
       u'ind_cno_fin_ult1', u'ind_ctju_fin_ult1', u'ind_ctma_fin_ult1',
       u'ind_ctop_fin_ult1', u'ind_ctpp_fin_ult1', u'ind_deco_fin_ult1',
       u'ind_deme_fin_ult1', u'ind_dela_fin_ult1', u'ind_ecue_fin_ult1',
       u'ind_fond_fin_ult1', u'ind_hip_fin_ult1', u'ind_plan_fin_ult1',
       u'ind_pres_fin_ult1', u'ind_reca_fin_ult1', u'ind_tjcr_fin_ult1',
       u'ind_valo_fin_ult1', u'ind_viv_fin_ult1', u'ind_nomina_ult1',
       u'ind_nom_pens_ult1', u'ind_recibo_ult1', u'targets_str',
       u'targets_logdec', u'targets_dec_g0', u'targets_dec

In [21]:
X_ym.columns

Index([u'age', u'antiguedad', u'canal_entrada', u'conyuemp', u'fecha_alta',
       u'fecha_dato', u'ind_actividad_cliente', u'ind_ahor_fin_ult1',
       u'ind_aval_fin_ult1', u'ind_cco_fin_ult1', u'ind_cder_fin_ult1',
       u'ind_cno_fin_ult1', u'ind_ctju_fin_ult1', u'ind_ctma_fin_ult1',
       u'ind_ctop_fin_ult1', u'ind_ctpp_fin_ult1', u'ind_deco_fin_ult1',
       u'ind_dela_fin_ult1', u'ind_deme_fin_ult1', u'ind_ecue_fin_ult1',
       u'ind_empleado', u'ind_fond_fin_ult1', u'ind_hip_fin_ult1',
       u'ind_nom_pens_ult1', u'ind_nomina_ult1', u'ind_nuevo',
       u'ind_plan_fin_ult1', u'ind_pres_fin_ult1', u'ind_reca_fin_ult1',
       u'ind_recibo_ult1', u'ind_tjcr_fin_ult1', u'ind_valo_fin_ult1',
       u'ind_viv_fin_ult1', u'indext', u'indfall', u'indrel', u'indrel_1mes',
       u'indresi', u'ncodpers', u'nomprov', u'pais_residencia', u'renta',
       u'segmento', u'sexo', u'tiprel_1mes', u'ult_fec_cli_1t',
       u'targets_str_1', u'targets_logdec_1', u'targets_dec_g0_1',
       

In [36]:
X_ym.index = X.index

In [45]:
from utils import dummies_to_decimal
from dataset2 import TARGET_GROUP_DEC_LABELS

In [62]:
label_index=1

def compute_diffs(df1, df2, label_index):
    compute_targets_diff(df1, df2, label_index)
    compute_targets_group_diff(df1, df2, label_index)

    
def compute_targets_diff(df1, df2, label_index):
    field_name = 'targets_diff_%i' % label_index
    df1.loc[:, field_name] = df1[TARGET_LABELS].apply(dummies_to_decimal, axis=1) - df2[TARGET_LABELS].apply(dummies_to_decimal, axis=1)

    
def compute_targets_group_diff(df1, df2, label_index):
    for label1 in TARGET_GROUP_DEC_LABELS:
        if label_index > 0:
            label2 = label1 + '_%i' % label_index
        field_name = label1 + '_diff_%i' % label_index
        df1.loc[:, field_name] = df1[label1] - df2[label2]
    

X_ = X.copy()
compute_diffs(X_, X_ym, 1)

In [55]:
cols = list(set(X_ym.columns) - set(FEATURES_NAMES + ['fecha_alta', 'fecha_dato', 'ncodpers'] + TARGET_LABELS))

In [65]:
for c in cols:
    X_.loc[:, c] = X_ym.loc[:, c]

In [69]:
X_.columns.values

array(['fecha_dato', 'ncodpers', 'ind_empleado', 'pais_residencia', 'sexo',
       'age', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel',
       'ult_fec_cli_1t', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext',
       'conyuemp', 'canal_entrada', 'indfall', 'nomprov',
       'ind_actividad_cliente', 'renta', 'segmento', 'ind_ahor_fin_ult1',
       'ind_aval_fin_ult1', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1',
       'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1',
       'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1',
       'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 'ind_ecue_fin_ult1',
       'ind_fond_fin_ult1', 'ind_hip_fin_ult1', 'ind_plan_fin_ult1',
       'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1',
       'ind_valo_fin_ult1', 'ind_viv_fin_ult1', 'ind_nomina_ult1',
       'ind_nom_pens_ult1', 'ind_recibo_ult1', 'targets_str',
       'targets_logdec', 'targets_dec_g0', 'targets_dec_g1',
       'targets_dec_g2', 'targets_dec_g3', '

In [42]:
X_ym[['ncodpers', 'targets_str_1']].head()

,ncodpers,targets_str_1
310503,15936,001000000000000000000000
309780,18987,001010000001110001110111
309739,19021,000000000000000000100000
311087,19299,000010000001100001010001
312469,20842,001000000000100000000000


In [ ]:
X_train.columns

In [ ]:
from dataset2 import TARGET_GROUP_DEC_LABELS

In [ ]:
X_train[['targets_str', 'targets_logdec'] + TARGET_GROUP_DEC_LABELS + FEATURES_NAMES].head()

In [ ]:
X_train.unique()

In [ ]:
X_train[['fecha_dato', 'ncodpers',]].sort_values(['fecha_dato', 'ncodpers'])

In [ ]:
X = X_train
from dataset2 import TARGET_GROUP_DEC_LABELS, FEATURES_NAMES
from dataset2 import TRAIN_FILE_PATH, load_data2, minimal_clean_data_inplace, preprocess_data_inplace, process_features, process_targets

supp_yearmonths_list = [201505, 201604]

In [ ]:
ref_clients = X['ncodpers'].unique()
# for ym in supp_yearmonths_list:
ym = supp_yearmonths_list[0]
fname = TEST_FILE_PATH if ym == 201606 else TRAIN_FILE_PATH
X_ym = load_data2(fname, [ym])
minimal_clean_data_inplace(X_ym)
preprocess_data_inplace(X_ym)
# process_targets(X_ym)
process_features(X_ym)



In [ ]:
clients = X_ym['ncodpers'].unique()
common_clients = np.array(list(set(ref_clients) & set(clients)))
missing_clients = np.array(list(set(ref_clients) - set(clients)))
print common_clients.shape, missing_clients.shape

In [ ]:
print X[X['ncodpers'].isin(missing_clients)].shape
X[X['ncodpers'].isin(missing_clients)].sort_values(['ncodpers']).head()

In [ ]:
print X_ym[X_ym['ncodpers'].isin(missing_clients)].shape
X_ym[X_ym['ncodpers'].isin(missing_clients)].sort_values(['ncodpers']).head()

In [ ]:
def features_to_str(features):
    return '_'.join([str(int(f)) for f in features])

Idea is to search for a feature profiles of missing clients in the supplementary month and create a new account in the supplementary month.

In [ ]:
from utils import _to_ym_dec, _to_nb_months

ref_ym = 201605
delta_months = _to_nb_months(_to_ym_dec(ref_ym) - _to_ym_dec(ym))
print delta_months

In [ ]:
X_ym_ = X_ym.copy()


def update_seniority(row):
    row['antiguedad'] -= delta_months
    return row


fg = match_features_groups[0]
print "- Compute missing clients : "
clients = X_ym_['ncodpers'].unique()
common_clients = np.array(list(set(ref_clients) & set(clients)))
missing_clients = np.array(list(set(ref_clients) - set(clients)))
print common_clients.shape, missing_clients.shape
print "- Feature group : ", fg
missing_clients_mask = X['ncodpers'].isin(missing_clients)
x_features_str = X[missing_clients_mask][fg].apply(update_seniority, axis=1).apply(features_to_str, axis=1)
x_ym_features_str = X_ym_[fg].apply(features_to_str, axis=1)
print "- Match and add"

In [ ]:
common_x_ym_features_str = x_ym_features_str[x_ym_features_str.isin(x_features_str)]
common_x_ym_features_str = common_x_ym_features_str.sort_values()
common_x_features_str = x_features_str[x_features_str.isin(common_x_ym_features_str)].sort_values()

In [ ]:
print common_x_ym_features_str.shape, common_x_features_str.shape

In [ ]:
gb = common_x_features_str.groupby(common_x_features_str)
gb_ym = common_x_ym_features_str.groupby(common_x_ym_features_str)

In [ ]:
common_x_ym_indices = gb_ym.apply(lambda x: x.index)
common_x_ym_indices.head()

In [ ]:
feature_index_map = gb.transform(lambda x: common_x_ym_indices[x])
print feature_index_map.shape
feature_index_map.head()

In [ ]:
set(X.columns) - set(FEATURES_NAMES) - set(TARGET_LABELS)

In [ ]:
supp_data_df = X.loc[common_x_features_str.index, ['ncodpers', 'fecha_alta', 'fecha_dato'] + FEATURES_NAMES].copy()
supp_data_df.head()

In [ ]:
supp_data_df.loc[:, 'fecha_dato'] = X_ym['fecha_dato'].unique()[0]
supp_data_df.loc[:, 'antiguedad'] -= delta_months
supp_data_df.head()

In [ ]:
from time import time

for t in TARGET_LABELS:
    supp_data_df[t] = 0

tic = time()
for index, value in feature_index_map.iteritems():
    targets = X_ym.loc[value, TARGET_LABELS].mean().apply(lambda x: int(np.ceil(x)))
    supp_data_df.loc[index, TARGET_LABELS] = targets
print time() - tic

In [ ]:
# supp_data_df[supp_data_df['ncodpers']==1284468].head()
supp_data_df.head()

In [ ]:
X_ym_ = pd.concat([X_ym_, supp_data_df], ignore_index=True)
X_ym_[X_ym_['ncodpers']==1284468]

In [ ]:
(X_ym_['ncodpers'].value_counts() > 1).any()

In [ ]:
from time import time

# def _match_replace_clients(ref_data, out_data):
#     _X, _x_features_str = ref_data
#     _X_ym_, _x_ym_features_str = out_data
    
#     _x_ym_features_str = _x_ym_features_str.drop_duplicates()
    
#     _X_sorted = _x_features_str.sort_values()
#     _X_ym_sorted = _x_ym_features_str.sort_values()
    
#     print _X_sorted.shape, _X_ym_sorted.shape
    
#     tic = time()
#     for index in _X_sorted.index:
#         m = _X_ym_sorted.eq(_X_sorted[index])
#         if m.any():
#             _X_ym_sorted_m = _X_ym_sorted[m]
#             # Choose randomly an index from matches and assign ncodpers to this line
#             ll = len(_X_ym_sorted_m.index)
#             ii = _X_ym_sorted_m.index[np.random.randint(ll)]
#             logging.debug("Match found : %i <-> %i | Replace ncodpers : %i <-> %i" % (index, _X_ym_sorted_m.index[0], _X_ym_.loc[ii, 'ncodpers'], _X.loc[index, 'ncodpers']))
#             _X_ym_.loc[ii, 'ncodpers'] = _X.loc[index, 'ncodpers']

#     print time() - tic
    
    
def match_add_clients(ref_data, out_data):

    _X, x_features_str = ref_data
    _X_ym_, x_ym_features_str = out_data
    
    tic = time()
 
    common_x_ym_features_str = x_ym_features_str[x_ym_features_str.isin(x_features_str)]
    common_x_ym_features_str = common_x_ym_features_str.sort_values()
    common_x_features_str = x_features_str[x_features_str.isin(common_x_ym_features_str)].sort_values()
    gb = common_x_features_str.groupby(common_x_features_str)
    gb_ym = common_x_ym_features_str.groupby(common_x_ym_features_str)
    
    common_x_ym_indices = gb_ym.apply(lambda x: x.index)
    feature_index_map = gb.transform(lambda x: common_x_ym_indices[x])

    supp_data_df = _X.loc[common_x_features_str.index, ['ncodpers', 'fecha_alta', 'fecha_dato'] + FEATURES_NAMES].copy()
    supp_data_df.loc[:, 'fecha_dato'] = _X_ym_['fecha_dato'].unique()[0]
    supp_data_df.loc[:, 'antiguedad'] -= delta_months

    for t in TARGET_LABELS:
        supp_data_df[t] = 0
    
    for index, value in feature_index_map.iteritems():
        targets = _X_ym_.loc[value, TARGET_LABELS].mean().apply(lambda x: int(np.ceil(x)))
        supp_data_df.loc[index, TARGET_LABELS] = targets
    
    _X_ym_ = pd.concat([_X_ym_, supp_data_df], ignore_index=True)
    print time() - tic
    
    assert not (_X_ym_['ncodpers'].value_counts() > 1).any(), "Something is wrong : {}".format(_X_ym_['ncodpers'].value_counts() > 1)
    return _X_ym_
    

In [ ]:
for i,f in enumerate(FEATURES_NAMES):
    print i, f

In [ ]:
NP_FEATURES_NAMES = np.array(FEATURES_NAMES)
NP_FEATURES_NAMES[[1, 2, 3, 6, 7, 8, 15]]

In [ ]:
NP_FEATURES_NAMES = np.array(FEATURES_NAMES)
match_features_groups = [
    FEATURES_NAMES,
    NP_FEATURES_NAMES[[0, 1, 2, 3, 4, 5, 6, 7, 8, 15, 16, 17, 18]],  # [u'ind_empleado', u'pais_residencia', u'sexo', u'age',  u'ind_nuevo', u'antiguedad', u'indrel', u'ult_fec_cli_1t', u'indrel_1mes', u'nomprov', u'ind_actividad_cliente', u'renta', u'segmento']
    NP_FEATURES_NAMES[[1, 2, 3, 4, 6, 7, 8, 15, 16, 17, 18]],  # [u'pais_residencia', u'sexo', u'age',  u'ind_nuevo', u'indrel', u'ult_fec_cli_1t', u'indrel_1mes', u'nomprov', u'ind_actividad_cliente', u'renta', u'segmento']
    NP_FEATURES_NAMES[[1, 2, 3, 4, 6, 7, 8, 15, 16, 18]],  # [u'pais_residencia', u'sexo', u'age', u'ind_nuevo', u'indrel', uult_fec_cli_1t', u'indrel_1mes', u'nomprov', u'ind_actividad_cliente', u'segmento']
    NP_FEATURES_NAMES[[1, 2, 3, 6, 7, 8, 15, 16, 18]],  # [u'pais_residencia', u'sexo', u'age', u'indrel', uult_fec_cli_1t', u'indrel_1mes', u'nomprov', u'ind_actividad_cliente', u'segmento']
    NP_FEATURES_NAMES[[1, 2, 3, 6, 7, 8, 15]],  # [u'pais_residencia', u'sexo', u'age', u'indrel', u'ult_fec_cli_1t', u'indrel_1mes', u'nomprov']
    ## Last try
    NP_FEATURES_NAMES[[1, 2, 3, 6, 15]],  # [u'pais_residencia', u'sexo', u'age', u'indrel', u'nomprov', u'segmento']
    NP_FEATURES_NAMES[[1, 2, 3, 15]],  # [u'pais_residencia', u'sexo', u'age', u'nomprov', u'segmento']
]

In [ ]:
fg

In [ ]:
logging.getLogger().setLevel(logging.INFO)

X_ym_ = X_ym.copy()

def update_seniority(row):
    row['antiguedad'] -= delta_months
    return row


for fg in match_features_groups:
    print "- Compute missing clients : "
    clients = X_ym_['ncodpers'].unique()
    common_clients = np.array(list(set(ref_clients) & set(clients)))
    missing_clients = np.array(list(set(ref_clients) - set(clients)))
    print common_clients.shape, missing_clients.shape
    print "- Feature group : ", fg
    missing_clients_mask = X['ncodpers'].isin(missing_clients)
    if TARGET_LABELS[5] in fg:
        x_features_str = X[missing_clients_mask][fg].apply(update_seniority, axis=1).apply(features_to_str, axis=1)
    else:
        x_features_str = X[missing_clients_mask][fg].apply(features_to_str, axis=1)
    x_ym_features_str = X_ym_[fg].apply(features_to_str, axis=1)
    print "- Match and add"
    X_ym_ = match_add_clients((X[missing_clients_mask], x_features_str), (X_ym_, x_ym_features_str))
    
    print X_ym_['ncodpers'].isin(missing_clients).sum(), X['ncodpers'].isin(missing_clients).sum()
    if X_ym_['ncodpers'].isin(missing_clients).sum() == X['ncodpers'].isin(missing_clients).sum():
        break
        

print "Compute missing clients : "
clients = X_ym_['ncodpers'].unique()
common_clients = np.array(list(set(ref_clients) & set(clients)))
missing_clients = np.array(list(set(ref_clients) - set(clients)))
print common_clients.shape, missing_clients.shape
        
# logging.getLogger().setLevel(logging.INFO)

In [ ]:
ref_clients.shape

In [ ]:
# If remains missing clients, setup them with zero targets 
if missing_clients.shape[0] > 0:
    print "There are still missing clients"
    missing_clients_mask = X['ncodpers'].isin(missing_clients)
    supp_data_df = X.loc[missing_clients_mask, ['ncodpers', 'fecha_alta', 'fecha_dato'] + FEATURES_NAMES].copy()
    supp_data_df.loc[:, 'fecha_dato'] = X_ym_['fecha_dato'].unique()[0]
    supp_data_df.loc[:, 'antiguedad'] -= delta_months

    for t in TARGET_LABELS:
        supp_data_df[t] = 0
        
    X_ym_ = pd.concat([X_ym_, supp_data_df], ignore_index=True)


In [ ]:
print "Compute missing clients : "
clients = X_ym_['ncodpers'].unique()
common_clients = np.array(list(set(ref_clients) & set(clients)))
missing_clients = np.array(list(set(ref_clients) - set(clients)))
print common_clients.shape, missing_clients.shape

Display loaded data

In [ ]:
# train_df[['fecha_dato', 'ncodpers'] + TARGET_LABELS_FRQ.tolist()].head(10)

Useful structures

In [ ]:
def get_common_clients(df1, mask1, mask2, df2=None):
    active_clients1 = df1[mask1]['ncodpers'].unique()
    if df2 is not None:
        active_clients2 = df2[mask2]['ncodpers'].unique()
    else:
        active_clients2 = df1[mask2]['ncodpers'].unique()
    active_clients = list(set(active_clients1) & set(active_clients2)) 
    
    if df2 is not None:
        return df1['ncodpers'].isin(active_clients), df2['ncodpers'].isin(active_clients)
    return df1['ncodpers'].isin(active_clients)

In [ ]:
months_ym_map = {}
# months = list(set(train_df['fecha_dato'].unique()) | set(val_df['fecha_dato'].unique()))
months = train_df['fecha_dato'].unique()
for m in months:
    months_ym_map[to_yearmonth(m)] = m

        
train_months = train_df['fecha_dato'].unique()
# val_months = val_df['fecha_dato'].unique()
    

### Train a model

In [ ]:
from utils import get_added_products, remove_last_choice, apk, map7_score
from visualization import visualize_train_test, visualize_folds, compare_two_datasets, compare_folds, compare_folds2

In [ ]:
target_features = ['targets_diff', 'targets_logdiff', 'targets_logcount2_diff', 'targets_logcount2', 'targets_logcount1', 'targets_logDec']
TARGET_LABELS_FRQ_PREV = [c + '_prev' for c in TARGET_LABELS_FRQ]

In [ ]:
def get_XY(current_month, df1, next_year_month, df2, months_ym_map):
    month_mask = df1['fecha_dato'] == months_ym_map[current_month]
    next_year_month_mask = df2['fecha_dato'] == months_ym_map[next_year_month]
    next_year_prev_month_mask = df2['fecha_dato'] == months_ym_map[next_year_month - 1]
    
    # get common clients from df1 at this month and df2 at next year month
    common_clients_mask1, common_clients_mask2 = get_common_clients(df1, month_mask, next_year_month_mask, df2)
    common_clients_mask2, common_clients_mask3 = get_common_clients(df2, common_clients_mask2 & next_year_month_mask, next_year_prev_month_mask, df2)
        
    c1 = df1[common_clients_mask1 & month_mask]['ncodpers'].values
    c2 = df2[common_clients_mask2 & next_year_month_mask]['ncodpers'].values
    c3 = df2[common_clients_mask3 & next_year_prev_month_mask]['ncodpers'].values
    assert (c1 == c2).all() and (c2 == c3).all(), "Problem with common clients" 
    
    X = df1[common_clients_mask1 & month_mask][['ncodpers', 'fecha_dato'] + target_features + features + TARGET_LABELS_FRQ.tolist()]            
   
    if TARGET_LABELS[0] in df2.columns and TARGET_LABELS_DIFF[0] in df2.columns and not df2[next_year_month_mask][TARGET_LABELS].isnull().all().all():
        Y = df2[common_clients_mask2 & next_year_month_mask][['ncodpers', 'fecha_dato', 'targets_str', 'lc_targets_str', 'targets_diff'] + TARGET_LABELS + TARGET_LABELS_DIFF.tolist()]    
        assert (X['ncodpers'].values == Y['ncodpers'].values).all(), "There is a problem in alignment"
        Y.index = X.index                
    else:
        Y = None
        
    if TARGET_LABELS_FRQ[0] in df2.columns and not df2[next_year_prev_month_mask][TARGET_LABELS].isnull().all().all():
        # Add TARGET_LABELS_FRQ from previous month to X:
        target_labels_frq = df2[common_clients_mask3 & next_year_prev_month_mask][['ncodpers'] + TARGET_LABELS_FRQ.tolist()]
        assert (X['ncodpers'].values == target_labels_frq['ncodpers'].values).all(), "There is a problem in alignment"
        target_labels_frq = target_labels_frq[TARGET_LABELS_FRQ]
        target_labels_frq.columns = TARGET_LABELS_FRQ_PREV
        target_labels_frq.index = X.index
        X = pd.concat([X, target_labels_frq], axis=1)        

    
    if LC_TARGET_LABELS[0] in df2.columns:
        clients_last_choice = df2[common_clients_mask2 & next_year_month_mask][['ncodpers', 'fecha_dato', 'targets_str'] + LC_TARGET_LABELS.tolist()]
    else:
        clients_last_choice = None
        
    return X, Y, clients_last_choice


In [ ]:
current_month = 201505
next_year_month = current_month + 100

df1 = train_df if months_ym_map[current_month] in train_months else val_df
#df1 = train_df
df2 = train_df if months_ym_map[next_year_month] in train_months else val_df
#df2 = train_df

X, Y, clients_last_choice = get_XY(current_month, df1, next_year_month, df2, months_ym_map)

In [ ]:
assert (X['ncodpers'].values == Y['ncodpers'].values).all(), "WTF"
assert (X['ncodpers'].values == clients_last_choice['ncodpers'].values).all(), "WTF"

In [ ]:
print X.shape
X.head(10)

In [ ]:
print Y.shape
Y[Y['targets_diff'] > 0][['fecha_dato', 'ncodpers', 'targets_str', 'lc_targets_str'] + TARGET_LABELS_DIFF.tolist() ].head(10)

In [ ]:
print clients_last_choice.shape
clients_last_choice.head(10)

## Another train/predict + CV implementation

### Input

- `X` : `[nb_samples, nb_features]` shaped pd.DataFrame
    - `features_masks_list` : `{fm1_name: features_mask_1, fm2_name: features_mask_2, ...]` with `features_mask_i` is a list of feature column names. They can oversect.
    
- `Y` : `[nb_samples, nb_labels]` shaped pd.DataFrame
    - `labels_masks_list` : `{lm1_name: labels_mask_1, lm2_name: labels_mask_2, ...}` with `labels_mask_i` is a list of labels column names. They can oversect.

- `samples_masks_list` : `[samples_mask_1, samples_mask_2, ...]` with samples_mask_i is a function to produce a boolean pd.DataFrame . Used only for training. 


- Set of models `models` : list of functions to create a model, e.g. `[create_RF, create_NN, create_GBT]`


### Training phase




In [ ]:
from collections import defaultdict
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Merge
from keras.utils import np_utils

from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
mask = Y['targets_diff'] > 0
targets_index_counts = np.zeros((len(TARGET_LABELS)))
for i, c in enumerate(TARGET_LABELS):
    s = (Y[mask][c] > 0).sum()
    targets_index_counts[i] = s

targets_index_counts = pd.Series(targets_index_counts)
targets_index_counts.plot.bar()

In [ ]:
samples_masks_list = [
#     lambda x, y:  ~(x['targets_diff'].isin([0])) | ~(y['targets_diff'].isin([0])), 
#     lambda x, y:  (x['targets_diff'] > 0) | (y['targets_diff'] > 0), 
    lambda x, y:  (y['targets_diff'] > 0), 
]

features_masks_dict = {
    'fm_all': features + target_features + TARGET_LABELS_FRQ.tolist() + TARGET_LABELS_FRQ_PREV,
#     'fm0': features + TARGET_LABELS_FRQ.tolist(),
#     'fm1': ['pais_residencia', 'sexo', 'age', 'ind_nuevo', 'segmento', 'ind_empleado', 'ind_actividad_cliente', 'indresi'],
#     'fm1': TARGET_LABELS_FRQ_PREV,
#     'fm2': target_features,
#     'fm3': ['pais_residencia', 'sexo', 'age', 'segmento', 'renta'],
#     'fm4': ['pais_residencia', 'sexo', 'age', 'renta', 'targets_logdiff', 'targets_logcount2_diff','targets_logcount2','targets_logcount1'],
#     'fm5': ['nomprov', 'ind_nuevo', 'renta', 'ind_actividad_cliente', 'canal_entrada'],
#     'fm6': TARGET_LABELS_FRQ,
}

In [ ]:
# https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/

def create_RF(input_shape, output_shape):        
    # https://www.analyticsvidhya.com/blog/2015/06/tuning-random-forest-model/
    return RandomForestClassifier(n_estimators=100, 
#                                   min_samples_split=100,
#                                   min_samples_leaf=25,
#                                   max_depth=10
                                  max_features=1.0, 
#                                   oob_score=True,
#                                   bootstrap=True,
                                  n_jobs=-1
                                 )

def create_ET(input_shape, output_shape):
    return ExtraTreesClassifier(n_estimators=100,
#                                   min_samples_leaf=25,
#                                   max_depth=10
                                  max_features=1.0, 
                                  oob_score=True,
                                  bootstrap=True,
                                  n_jobs=-1

                               )

def create_GB(input_shape, output_shape):
    return GradientBoostingClassifier(n_estimators=75)


def create_NN0(input_shape, output_shape):
        
    assert len(input_shape) == 2, "Input shape should be 2D"
    assert len(output_shape) == 2, "Input shape should be 2D"
    n_features = input_shape[1]
    output_dim = output_shape[1]
    
    def create_model(input_dim=n_features, output_dim=output_dim):
        model = Sequential()
        model.add(Dense(30, init='uniform', input_shape=(input_dim,), activation='relu'))
        model.add(Dropout(0.15))
#         model.add(Dense(output_dim, activation='sigmoid'))
        model.add(Dense(output_dim, activation='softmax'))
#         model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
        model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])
        return model

    return KerasClassifier(build_fn=create_model, nb_epoch=200, batch_size=2000, verbose=0)
    

models_dict = {
    'rf': create_RF,
    'et': create_ET,
    'gb': create_GB,
#     'nn0': create_NN0
}

In [ ]:
print mask.sum()
print targets_index_counts[targets_index_counts > 100].index

In [ ]:
NP_TARGET_LABELS = np.array(TARGET_LABELS)
target_labels = NP_TARGET_LABELS

common_groups = [
    [2, 4, 7, 8, 11, 12, 13, 15, 17, 18, 19, 21, 22, 23],
#     [2, 4],    
#     [2, 6, 7, 8],
#     [2, 18, 23, 12], 
#     [21, 22],
#     [2, 12, 18],
#     [2, 12, 23],
#     [2, 18, 23],
#     [18, 23, 21, 22],
#     [21, 23, 22, 4],
#     [3, 4], 
#     [22, 7, 8, 23],
#     [0, 1, 14, 15, 17]
]

common_groups += [[i] for i in [2, 4, 7, 8, 11, 12, 13, 15, 17, 18, 19, 21, 22, 23]]



def flatten(array):
    out = []
    for item in array:
        out += item
    return out

others = list(set(range(24)) - set(flatten(common_groups)))

# for i, a in enumerate(zip(TARGET_LABELS2, TARGET_LABELS)):
#     print i, a
    
s = set({})
labels_masks_dict = {}
for i, g in enumerate(common_groups):
    print 'lm_%i' % i, " <=> ", g, "<==>", TARGET_LABELS2[g]
    labels_masks_dict['lm_%i' % i] = target_labels[g]
    s |= set(g)
    
# print 'lm_others', "<=>", others, "<==>", TARGET_LABELS2[others]
# labels_masks_dict['lm_others'] = target_labels[others]
# s |= set(others)

# assert len(s) == len(target_labels), "Sum is not equal 24, s=%i" % s
print labels_masks_dict

In [ ]:
# {model_name: [(samples_mask_code, features_mask_name, labels_mask_name), ...]}
models_pipelines = {
    'gb' : [(None, None, key) for key in labels_masks_dict if len(labels_masks_dict[key]) == 1],    
    'rf' : [(None, None, key) for key in labels_masks_dict if len(labels_masks_dict[key]) > 1],
    'et' : [(None, None, key) for key in labels_masks_dict if len(labels_masks_dict[key]) > 1],
}
models_pipelines

In [ ]:
from trainval import train_all, predict_all, probas_to_indices, score_estimators
from utils import map7_score0

In [ ]:
ll = 140000
# ll = 1100

mask = (X.index.isin((X.index[:ll//3]))) | (X.index.isin((X.index[2*ll//3:])))

X1 = X[mask]
Y1 = Y[mask]
clc1 = clients_last_choice[mask]
print X1.shape, Y1.shape, clc1.shape

mask = X.index.isin(X.index[ll//3:2*ll//3])
X2 = X[mask]
Y2 = Y[mask]
clc2 = clients_last_choice[mask]
print X2.shape, Y2.shape, clc2.shape

In [ ]:
# res = Y1[labels_masks_dict['lm_0']].apply(dummies_to_decimal, axis=1)
# res = pd.get_dummies(res)
# res.head()

In [ ]:
from utils import dummies_to_decimal
from sklearn.preprocessing import StandardScaler


def prepare_to_fit(X_train, Y_train):    
    x_train = X_train.values
    x_train = StandardScaler().fit_transform(x_train)
    y_train = Y_train.apply(dummies_to_decimal, axis=1)
    y_train = pd.get_dummies(y_train)
    y_train = y_train.values    
    return x_train, y_train


def prepare_to_test(X_val, Y_val=None):
    x_val = X_val.values
    x_val = StandardScaler().fit_transform(x_val)
    if Y_val is not None:
        y_val = Y_val.apply(dummies_to_decimal, axis=1)
        y_val = pd.get_dummies(y_val)
        y_val = y_val.values 
    else:
        y_val = None
    return x_val, y_val


def probas_to_labels_probas(y_probas, class_indices, labels):
    l = len(labels)
    out = np.zeros((len(y_probas), l))
    i = 0
    for probas in y_probas:
        if np.sum(probas) > 0:
            pr = np.zeros((l,))
            for index, p in zip(class_indices, probas):
                dummies_str = decimal_to_dummies(index, l)
                pr += p * np.array([float(v) for v in dummies_str])
            out[i, :] = pr    
        i += 1
    return out
    

In [ ]:
_kwargs = {
    'samples_masks_list': samples_masks_list, 
    'features_masks_dict': features_masks_dict, 
    'labels_masks_dict': labels_masks_dict, 
    'models_dict': models_dict,
    'labels': target_labels,
    'transform_proba_func': probas_to_indices,
#     'prepare_to_fit_func': prepare_to_fit,
#     'prepare_to_test_func': prepare_to_test,   
#     'probas_to_labels_probas_func': probas_to_labels_probas,
    'threshold': 0.15,
    'n_highest': 7,
    'mode': 'sum',
    'verbose': True,
    'models_pipelines': models_pipelines,
    'return_probas': True
}

In [ ]:
estimators = train_all(X1, Y1, **_kwargs)

#print estimators

In [ ]:
accuracies = defaultdict(list)
for e in estimators:
    accuracies[e[0][2]].append(e[2])

mean_accuracy = {}
for key in accuracies:
    accuracy_list = accuracies[key]
    mean_accuracy[key] = sum(accuracy_list)/len(accuracy_list)
    
mean_accuracy

In [ ]:
_ = score_estimators(estimators, X2, Y2, **_kwargs)

In [ ]:
y_preds, Y_probas = predict_all(estimators, X2, **_kwargs)
#print y_preds[:5]

In [ ]:
print y_preds[:5]
Y_probas.head()

In [ ]:
#labels_masks_dict['lm_0'], common_groups[0]

In [ ]:
y_val = targets_str_to_indices(Y2[target_labels].values)
# y_val = targets_str_to_indices(Y2[labels_masks_dict['lm_0']].values, index_map=common_groups[0])
#print y_val[:100]
#print y_preds[:100]

In [ ]:
#np.unique(y_preds)

In [ ]:
logging.info("- Compute max map7 score")
map7_score(y_val, y_val, clc2[LC_TARGET_LABELS].values)
# map7_score0(y_val, y_val)
logging.info("- Compute map7 score")
map7_score(y_val, y_preds, clc2[LC_TARGET_LABELS].values)
# map7_score0(y_val, y_preds)


On columns lm_0=['ind_cco_fin_ult1', 'ind_cder_fin_ult1', 'ind_cno_fin_ult1','ind_ctju_fin_ult1']

- Feature mask: all : 'fm6', 'fm4', 'fm5', 'fm2', 'fm3', 'fm0', 'fm1'
- threshold = 0.0

Model | Map@7 | Max Map@7 | Labels mask | Samples mask
--- | --- | --- | ---
et | 0.007074370518592629 | 0.0075787893947 | lm_0 | all 



- Feature mask: fm0, fm1, fm3, fm4, fm5
- threshold = 0.0

Model | Map@7 | Max Map@7 | Labels mask | Samples mask
--- | --- | --- | ---
rf + et + gb | 0.006920126730031681 | 0.0075787893947 | lm_0 | all
rf | 0.0068805235951309 | 0.0075787893947 | lm_0 | all
et | 0.006936801734200433 | 0.0075787893947 | lm_0 | all 
gb | 0.0068805235951309 | 0.0075787893947 | lm_0 | all


- Feature mask: fm0, fm1

Model | Map@7 | Max Map@7 | Labels mask | Samples mask
--- | --- | --- | ---
rf + et + gb | 0.004627313656828414 | 0.0075787893947 | lm_0 | all
rf | 0.004664832416208104 | 0.0075787893947 | lm_0 | all
et | 0.004952476238119059 | 0.0075787893947 | lm_0 | all 
gb | 0.004489744872436218 | 0.0075787893947 | lm_0 | all

- Features mask: fm0

Model | Map@7 | Max Map@7 | Labels mask | Samples mask
--- | --- | --- | --- | ---
rf + et + gb | 0.0021010505252626313 | 0.0075787893947 | lm_0 | all
rf + et | 0.001950975487743872 | 0.0075787893947 | lm_0 | all
rf | 0.001550775387693847 | 0.0075787893947 | lm_0 | all
gb | 0.0013006503251625813 | 0.0075787893947 | lm_0 | all
et | 0.0017008504252126063 | 0.0075787893947 | lm_0 | all
et |  0.0014007003501750874 | 0.0075787893947 | lm_0 | x>0 or y>0
rf |  0.0008254127063531766 | 0.0075787893947 | lm_0 | .

--------------------------------------

0.021295269099703414 (GB on 'all')

0.021271936353906683 (RF tunning)

0.021668245671284416 (RF tunning)

0.02136609107928888

0.0211362663776694

In [ ]:
# print labels_masks_dict[estimators[0][0][1]]
# print estimators[0][1].classes_
# print estimators[0][1].n_classes_
# print estimators[0][1].n_features_
# print estimators[0][1].n_outputs_
# print estimators[0][1].estimators_

In [ ]:
from utils import targets_to_labels, targets_indices_to_labels, remove_last_choice

In [ ]:
limit = 25
count = 0

not_predicted_predicted = defaultdict(int)
for last_choice, targets, products, proba in zip(clc2[LC_TARGET_LABELS].values, y_val, y_preds, Y_probas.values):
    added_products = remove_last_choice(targets, last_choice)
    predictions = remove_last_choice(products, last_choice)
#     print "---", count, last_choice
#     print targets, '->', added_products
#     print products, '->', predictions
#     if count == 3:
#         break
    
    if len(added_products) == 0:
        continue
        
    if len(set(added_products) & set(predictions)) > 0:
#         print "Predicted : ", added_products, predictions
#         print set(added_products) & set(predictions)
        continue

    count += 1
    if count < limit:
        print "--- Count = ", count
        print targets_indices_to_labels(added_products, TARGET_LABELS2)#, targets_indices_to_labels(targets, TARGET_LABELS2)
        print targets_indices_to_labels(predictions, TARGET_LABELS2)#, targets_indices_to_labels(products, TARGET_LABELS2)#, proba
    
    for p in added_products:
        not_predicted_predicted[TARGET_LABELS2[p]] += 1
    

In [ ]:
print not_predicted_predicted, y_val.shape[0]

In [ ]:
#print y_probas[:10, target_groups[0]]
#print Y[np.array(TARGET_LABELS)[target_groups[0]]].head(10)

### Run KFold Cross-validation 

In [ ]:
from trainval import cross_val_score0, cross_val_score

In [ ]:
# Unitary run
nb_folds = 5
results = cross_val_score((X, Y, clients_last_choice[LC_TARGET_LABELS].values), nb_folds=nb_folds, **_kwargs)

print "Cross-Validation \n %i | %f | %f | %f | %f | %.5f " % (nb_folds, results.min(), results.mean(), np.median(results), results.max(), results.std())


In [ ]:
from itertools import combinations
# CV on various combinations :

_samples_masks_list = [
    'all',
#     lambda x, y:  ~(x['targets_diff'].isin([0])), 
#     lambda x, y:  x['targets_diff'] > 0, 
#     lambda x, y:  x['targets_diff'] < 0, 
    lambda x, y:  ~(x['targets_diff'].isin([0])) | ~(y['targets_diff'].isin([0])), 
    lambda x, y:  (x['targets_diff'] > 0) | (y['targets_diff'] > 0), 
#     lambda x, y:  (x['targets_diff'] < 0) | (y['targets_diff'] < 0), 
#     lambda x, y:  (y['targets_diff'] > 0), 
#     lambda x, y:  y['targets_diff'] < 0, 
]

_features_masks_dict = {
    'fm0': features + target_features + TARGET_LABELS_FRQ.tolist() + TARGET_LABELS_FRQ_PREV,
    'fm1': ['pais_residencia', 'sexo', 'age', 'ind_nuevo', 'segmento', 'ind_empleado', 'ind_actividad_cliente', 'indresi'],
    'fm2': target_features,
    'fm3': ['pais_residencia', 'sexo', 'age', 'segmento', 'renta'],
    'fm4': ['pais_residencia', 'sexo', 'age', 'renta', 'targets_logdiff', 'targets_logcount2_diff','targets_logcount2','targets_logcount1'],
    'fm5': ['nomprov', 'ind_nuevo', 'renta', 'ind_actividad_cliente', 'canal_entrada'],
    'fm6': TARGET_LABELS_FRQ,
}

_models_dict = {
    'rf': create_RF,
    'et': create_ET,
    'gb': create_GB,
}

_labels_masks_dict = {
    'lm_0': labels_masks_dict['lm_0']
}

nb_folds = 5

def BruteForceSearchCV():
    
    def get_models_combinations(items):
        combins = list(combinations(items, 1))
        combins += list(combinations(items, len(items)))
        return combins
    
    def get_combinations(items):
        combins = list(combinations(items, 1))
        for i in range(2, len(items)+1):
            combins += list(combinations(items, i))
        return combins
    
    def get_items(items):
        out = [[items[0],], ]
        for i in items[1:]:
            tmp = list(out[-1])
            tmp.append(i)
            out.append(tmp)        
        return out

    
    _labels_masks_combinations = get_items(sorted(_labels_masks_dict.keys()))
    _features_masks_combinations = get_items(sorted(_features_masks_dict.keys()))
    _models_combinations = get_models_combinations(_models_dict.keys())

    # Very big loop:
    for lm_keys in _labels_masks_combinations:
        __labels_masks_dict = {}
        for lm_key in lm_keys:
            __labels_masks_dict[lm_key] = _labels_masks_dict[lm_key]

        for i, sm in enumerate(_samples_masks_list):
            __samples_masks_list = [sm]
              
            for fm_keys in _features_masks_combinations:
                __features_masks_dict = {}
                for fm_key in fm_keys:
                    __features_masks_dict[fm_key] = _features_masks_dict[fm_key]
                    
                for m_keys in _models_combinations:
                    __models_dict = {}
                    for m_key in m_keys:
                        __models_dict[m_key] = _models_dict[m_key]
                    
                    print "\n\n---------------------------------------------------------------" 
                    print "--- PROCESS : ", __labels_masks_dict.keys(), i, __features_masks_dict.keys(), __models_dict.keys()
                    print "---------------------------------------------------------------\n" 
                    
                    __kwargs = {
                        'samples_masks_list': __samples_masks_list, 
                        'features_masks_dict': __features_masks_dict, 
                        'labels_masks_dict': __labels_masks_dict, 
                        'models_dict': __models_dict,
                        'labels': target_labels,
                        'transform_proba_func': probas_to_indices,
                        'prepare_to_fit_func': prepare_to_fit,
                        'prepare_to_test_func': prepare_to_test,   
                        'probas_to_labels_probas_func': probas_to_labels_probas,
                        'threshold': 0.0,
                        'n_highest': 7,
                        'mode': 'sum',
                        'verbose': False,
                        'return_probas': True
                    }
                    #  DEBUG : results = cross_val_score((X1, Y1, clc1[LC_TARGET_LABELS].values), nb_folds=nb_folds, **__kwargs)
                    results = cross_val_score((X, Y, clients_last_choice[LC_TARGET_LABELS].values), nb_folds=nb_folds, **__kwargs)
                    print "=> CV : ", results

In [ ]:
logging.getLogger().setLevel(logging.WARNING)
BruteForceSearchCV()
logging.getLogger().setLevel(logging.INFO)


### 201505 -> 201605 

Cross-Validation 
 5 | 0.014585 | 0.018385 | 0.019147 | 0.022227 | 0.00294 

Compute cross-validation across several months

In [ ]:
nb_folds = 3
yms = [201504, 201505]
#yms = [201505]

for ym in yms:
    logging.info("\n-------------------------")
    logging.info("- Process month : %s" % ym)
    logging.info("-------------------------\n")
    
    ym1 = ym + 100    
    df1 = train_df if months_ym_map[ym] in train_months else val_df
    df2 = train_df if months_ym_map[ym1] in train_months else val_df
    X, Y, clients_last_choice = get_XY(ym, df1, ym1, df2) 
    results = cross_val_score2((X, Y, clients_last_choice[LC_TARGET_LABELS].values), 
                                profiles=profiles,
                                nb_folds=nb_folds)
    print "Cross-Validation \n %i | %f | %f | %f | %.5f " % (nb_folds, results.min(), results.mean(), results.max(), results.std())

## Train model for predictions

In [ ]:
current_month = 201505
next_year_month = current_month + 100

df1 = train_df
#df1 = val_df
df2 = train_df #if months_ym_map[next_year_month] in train_months else val_df
#df2 = val_df

X, Y, clients_last_choice = get_XY(current_month, df1, next_year_month, df2, months_ym_map)

In [ ]:
estimators = train_all(X, Y, **_kwargs)

In [ ]:
y_preds, Y_probas = predict_all(estimators, X, **_kwargs)

Check score on the data 2016/05

In [ ]:
logging.info("- Compute map7 score")
print map7_score(y_val, y_preds, clients_last_choice[LC_TARGET_LABELS].values)
logging.info("- Compute max map7 score")
print map7_score(y_val, y_val, clients_last_choice[LC_TARGET_LABELS].values)

## Prediction for 2016/06

In [ ]:
from dataset import load_train_test

In [ ]:
full_train_df, test_df = load_train_test([201506])

In [ ]:
full_train_df.head()

In [ ]:
test_df.head()

In [ ]:
months_ym_map = {}
months = list(set(full_train_df['fecha_dato'].unique()) | set(test_df['fecha_dato'].unique()))
for m in months:
    months_ym_map[to_yearmonth(m)] = m
    
full_train_months = full_train_df['fecha_dato'].unique()
test_months = test_df['fecha_dato'].unique()

In [ ]:
current_month = 201506
next_year_month = current_month + 100

df1 = full_train_df
df2 = test_df
X, _, clients_last_choice = get_XY(current_month, df1, next_year_month, df2, months_ym_map)

In [ ]:
print X.shape, test_df.shape

In [ ]:
X.head(10)

In [ ]:
clients_last_choice.head(10)

In [ ]:
def get_submission(predicted_added_products, clients, clc, target_labels):
    added_products_col = []
    count = 0 
    for products, last_choice in zip(predicted_added_products, clc):
        predictions = remove_last_choice(products, last_choice)
        added_products_col.append(' '.join([target_labels[i] for i in predictions]))
        count+=1
        if count % 100000 == 0:
            logging.info("Elapsed : %i", count)
            
    out = pd.DataFrame(data={'ncodpers': clients, 'added_products': added_products_col}, columns=['ncodpers', 'added_products'])
    return out

In [ ]:
y_preds, Y_probas = predict_all(estimators, X, **_kwargs)

logging.info("- Get submission dataframe:")
clients = X['ncodpers'].values
#submission = get_submission(y_preds, clients, clients_last_choice[LC_TARGET_LABELS].values, TARGET_LABELS)

In [ ]:
submission = get_submission(y_preds, clients, clients_last_choice[LC_TARGET_LABELS].values, TARGET_LABELS)

In [ ]:
print submission.shape
submission.head()

In [ ]:
selected_estimators = []
for e in estimators:
    # estimators = [([features_mask_name, labels_mask_name, model_name], estimator_object, accuracy), ...]
    features_mask_name, labels_mask_name, model_name = e[0]
#     print features_mask_name, labels_mask_name, model_name
    if set(features_masks_dict[features_mask_name]).issubset(test_df.columns):
#         print "Append the estimator"
        selected_estimators.append(e)

In [ ]:
submission_clients = set(submission['ncodpers'].unique())
test_clients = set(test_df['ncodpers'].unique())
if submission_clients != test_clients:
    missing_clients = list(test_clients - submission_clients)
    missing_clients_mask = test_df['ncodpers'].isin(missing_clients)
    
    X1 = test_df[missing_clients_mask]
        
    y_preds, Y_probas = predict_all(selected_estimators, X1, **_kwargs)    
    submission2 = get_submission(y_preds, missing_clients, X1[LC_TARGET_LABELS].values, TARGET_LABELS)
    
#     submission = pd.concat([submission, submission2])

In [ ]:
submission2 = get_submission(y_preds, missing_clients, X1[LC_TARGET_LABELS].values, TARGET_LABELS)
#submission = pd.concat([submission, submission2])

In [ ]:
print submission2.shape
submission2.head()

Get submission DataFrame and write csv file

In [ ]:
print submission.shape
submission.head()

In [ ]:
from datetime import datetime
import csv

logging.info('- Generate submission')
submission_file = '../results/submission_' + \
                  str(datetime.now().strftime("%Y-%m-%d-%H-%M")) + \
                  '.csv'

submission.to_csv(submission_file, index=False, index_label=False)

In [ ]:
with open('../results/submission_2016-11-17-16-37.csv', 'r') as r:
    print r.readline()
    print r.readline()
    print r.readline()
    print r.readline()
    print r.readline()
    